# Домашнее задание


# 🎈🍾 🎉





Вы работаете в крупном онлайн-магазине, который реализует товары «всё для праздников». Вам необходимо провести ABC-анализ и выявить наиболее важные товары на основе объемов продаж и популярности. Эта информация составит план закупок на весь следующий год. 

**ABC-анализ** позволяет определить наиболее важные ресурсы компании на основе объемов продаж и прибыли.

В основе ABC-анализа лежит __«Принцип Парето»: 20% ресурсов приносят 80% прибыли__. Именно этот вид анализа позволяет выделить те самые  группы товаров, ресурсов, клиентов, поставщиков и т. п., которые и приносят компании основную прибыль.

__Задача анализа разделить товары на группы по принципу:__

- А — наиболее ценные, 20% — ассортимента (номенклатура); 80% — продаж
- В — промежуточные, 30% — ассортимента; 15% — продаж
- С — наименее ценные, 50% — ассортимента; 5% — продаж

## Задание 1

Сохраните все данные из таблицы `products` из PostgreSQL в дата-фрейм `products` и подготовьте данные к анализу.

1. Подгрузите данные для подключения
2. Подключите нужную библиотеку для работы с PostgreSQL
3. Создайте соединение с БД
4. Создайте запрос и выгрузите все данные из таблицы `products`
5. Положите данные в дата-фрейм `products`
6. Приведите данные к удобному типу: числа к числовому типу, идентификаторы — к текстовому


In [31]:
from postgresql_secret import pg_secret

In [32]:
pg_secret

{'db_host': '178.154.254.133',
 'db_name': 'skyeng',
 'db_password': 'readonlyuser',
 'db_port': '5432',
 'db_user': 'readonlyuser'}

In [33]:
import psycopg2

In [34]:
connection = psycopg2.connect(
                             database=pg_secret['db_name'],
                             user=pg_secret['db_user'],
                             password=pg_secret['db_password'],
                             host=pg_secret['db_host'],
                             port=pg_secret['db_port'],
                             )

cursor = connection.cursor()

In [35]:
import pandas as pd

query = "SELECT * FROM products"# напиши запрос, твоих знаний SQL точно хватит
cursor.execute(query) 
result = cursor.fetchall() 

cursor.description

columns = []
for desc in cursor.description:  
    columns.append(desc[0])

products = pd.DataFrame(result, columns=columns)  # Из списка кортежей сделаем датафрейм
products.head()

,invoice_id,stock_code,description,quantity,price
0,489805,17109B,FLOWER FAIRY INCENSE LAVENDER,2,1.69
1,489805,17109C,FLOWER FAIRY INCENSE ROSE,3,1.69
2,489805,21485,RED SPOT HEART HOT WATER BOTTLE,9,4.95
3,489805,21034,REX CASH+CARRY JUMBO SHOPPER,1,0.95
4,489805,21622,VINTAGE UNION JACK CUSHION COVER,25,4.95


In [36]:
#Приведите данные к удобному типу: числа к числовому типу, идентификаторы — к текстовому
products['quantity'] = products['quantity'].astype(int)
products['price'] = products['price'].astype(int)
products['invoice_id'] = products['invoice_id'].apply(str)
products['stock_code'] = products['stock_code'].apply(str)

## Задание 2

__Посчитайте, какой доход принес каждый продукт и сколько единиц каждого продукта было продано__

1. Посчитай общую сумму оплат для каждой строки счета
2. Получи общий доход и количество единиц по товарам
3. Объедини сгруппированные данные в один датафрейм со столбцами: `description`, `stock_code`, `quantity`, `total`

In [37]:
products['total'] = sum(products['price'])
products.head()

,invoice_id,stock_code,description,quantity,price,total
0,489805,17109B,FLOWER FAIRY INCENSE LAVENDER,2,1,2044788
1,489805,17109C,FLOWER FAIRY INCENSE ROSE,3,1,2044788
2,489805,21485,RED SPOT HEART HOT WATER BOTTLE,9,4,2044788
3,489805,21034,REX CASH+CARRY JUMBO SHOPPER,1,0,2044788
4,489805,21622,VINTAGE UNION JACK CUSHION COVER,25,4,2044788


In [78]:
Table1 = products.groupby(['stock_code', 'description'])['price'].sum().reset_index()
Table1.head()
# посчитай общих доход по каждому товару

,stock_code,description,price
0,10002,INFLATABLE POLITICAL GLOBE,0
1,10080,GROOVY CACTUS INFLATABLE,0
2,10109,BENDY COLOUR PENCILS,0
3,10120,DOGGY RUBBER,0
4,10123C,HEARTS WRAPPING TAPE,0


In [79]:
# посчитай общее количество проданных единиц по каждому товару
Table2 = products.groupby('description')['quantity'].sum().reset_index()
Table2.head()

,description,quantity
0,DOORMAT UNION JACK GUNS AND ROSES,167
1,3 STRIPEY MICE FELTCRAFT,662
2,4 PURPLE FLOCK DINNER CANDLES,324
3,50'S CHRISTMAS GIFT BAG LARGE,1885
4,ANIMAL STICKERS,385


In [80]:
# объедини общие данные в одну таблицу
TableGeneral=Table1.merge(Table2)
TableGeneral['total_price'] = sum(products['price'])
TableGeneral.head()

,stock_code,description,price,quantity,total_price
0,10002,INFLATABLE POLITICAL GLOBE,0,8479,2044788
1,10080,GROOVY CACTUS INFLATABLE,0,303,2044788
2,10109,BENDY COLOUR PENCILS,0,4,2044788
3,10120,DOGGY RUBBER,0,648,2044788
4,10123C,HEARTS WRAPPING TAPE,0,628,2044788


## Задание 3

__Найди нарастающий итог по процентам__

1. Отсортируй датафрейм по доходу в нисходящем порядке 
2. Найди процент дохода каждого товара в общем доходе 
3. Сделай нарастающий итог (running total) по процентам каждого товара, для этого найди в интернете и примени метод для расчета кумулятивной суммы в `pandas`. 


Нарастающий итог — это представление данных, в котором мы видим динамику нарастания или убывания суммы от начала отсчета до конца. Получаем мы эту кумулятивную сумму, прибавлением поочередно каждого следующего элемента к сумме предыдущих.  



In [81]:
# отсортируй дата-фрейм (большие вверху, маленькие внизу)
TableGeneral.sort_values(by='price', ascending = False, inplace=True)
print(TableGeneral)

     stock_code                          description   price  quantity  \
5309          M                               Manual  145964      9391   
5311       POST                              POSTAGE   53968      5235   
1862      22423             REGENCY CAKESTAND 3 TIER   38941     24139   
5306         C2                             CARRIAGE   12528       249   
5308        DOT                       DOTCOM POSTAGE   11901        16   
...         ...                                  ...     ...       ...   
2922      23281       FOLDING BUTTERFLY MIRROR RED         0      1252   
2921      23280             FOLDING MIRROR HOT PINK        0       156   
4705      85072       OCCASIONS GIFT CARD ASSORTMENT       0       120   
2914      23271  CHRISTMAS TABLE CANDLE SILVER SPIKE       0        16   
0         10002          INFLATABLE POLITICAL GLOBE        0      8479   

      total_price  
5309      2044788  
5311      2044788  
1862      2044788  
5306      2044788  
5308      2

In [82]:
# найди процент дохода для каждого товара 

#(искомая часть / целое число) * 100

TableGeneral['%_price'] = (TableGeneral['price'])/(TableGeneral['total_price'])*100
print(TableGeneral)

     stock_code                          description   price  quantity  \
5309          M                               Manual  145964      9391   
5311       POST                              POSTAGE   53968      5235   
1862      22423             REGENCY CAKESTAND 3 TIER   38941     24139   
5306         C2                             CARRIAGE   12528       249   
5308        DOT                       DOTCOM POSTAGE   11901        16   
...         ...                                  ...     ...       ...   
2922      23281       FOLDING BUTTERFLY MIRROR RED         0      1252   
2921      23280             FOLDING MIRROR HOT PINK        0       156   
4705      85072       OCCASIONS GIFT CARD ASSORTMENT       0       120   
2914      23271  CHRISTMAS TABLE CANDLE SILVER SPIKE       0        16   
0         10002          INFLATABLE POLITICAL GLOBE        0      8479   

      total_price   %_price  
5309      2044788  7.138344  
5311      2044788  2.639296  
1862      2044788  1.

In [83]:
# используй метод pandas для кумулятивной суммы
TableGeneral['cumsum_price'] =  TableGeneral['%_price'].cumsum()
print(TableGeneral)

     stock_code                          description   price  quantity  \
5309          M                               Manual  145964      9391   
5311       POST                              POSTAGE   53968      5235   
1862      22423             REGENCY CAKESTAND 3 TIER   38941     24139   
5306         C2                             CARRIAGE   12528       249   
5308        DOT                       DOTCOM POSTAGE   11901        16   
...         ...                                  ...     ...       ...   
2922      23281       FOLDING BUTTERFLY MIRROR RED         0      1252   
2921      23280             FOLDING MIRROR HOT PINK        0       156   
4705      85072       OCCASIONS GIFT CARD ASSORTMENT       0       120   
2914      23271  CHRISTMAS TABLE CANDLE SILVER SPIKE       0        16   
0         10002          INFLATABLE POLITICAL GLOBE        0      8479   

      total_price   %_price  cumsum_price  
5309      2044788  7.138344      7.138344  
5311      2044788  2.63

## Задание 4

__Пометь товары метками `'A'`, `'B'`, `'C'` в зависимости от нарастающего итога__

1. Напиши функцию, которая будет помечать товары с нарастающим итогом ниже или равно 80% как группу `'A'`, от 80%-95% — как `'B'` и остальные как `'C'`

In [84]:
def revenue(x):
    # допиши функцию
    if x <= 80:
        return 'A'
    elif x in range(80,95):
        return 'B'
    else:
        return 'C'
# создай новый столбец revenue с помощью функции
TableGeneral['revenue'] = TableGeneral['cumsum_price'].apply(revenue) 
TableGeneral

,stock_code,description,price,quantity,total_price,%_price,cumsum_price,revenue
5309,M,Manual,145964,9391,2044788,7.138344,7.138344,A
5311,POST,POSTAGE,53968,5235,2044788,2.639296,9.777640,A
1862,22423,REGENCY CAKESTAND 3 TIER,38941,24139,2044788,1.904403,11.682042,A
5306,C2,CARRIAGE,12528,249,2044788,0.612680,12.294722,A
5308,DOT,DOTCOM POSTAGE,11901,16,2044788,0.582016,12.876738,A
...,...,...,...,...,...,...,...,...
2922,23281,FOLDING BUTTERFLY MIRROR RED,0,1252,2044788,0.000000,100.000000,C
2921,23280,FOLDING MIRROR HOT PINK,0,156,2044788,0.000000,100.000000,C
4705,85072,OCCASIONS GIFT CARD ASSORTMENT,0,120,2044788,0.000000,100.000000,C
2914,23271,CHRISTMAS TABLE CANDLE SILVER SPIKE,0,16,2044788,0.000000,100.000000,C


## Задание 5

__Найди нарастающий итог по количеству товарных единиц и пометь товары метками `'A'`, `'B'`, `'C'`, отмечая тем самым популярность товара__

1. Отсортируй предыдущий датафрейм с данными о доходе теперь по количеству товаров
2. Найди процент каждого товара в общем количестве
3. Найди нарастающий итог по количеству
4. Напиши функцию с теми же параметрами, что и у дохода
5. Отметь группу

In [85]:
# проведи те же операции, что в двух предыдущих заданиях, для оценки популярности товаров
TableGeneral['total_quantity'] = sum(products['quantity'])

TableGeneral.sort_values(by='quantity', ascending = False, inplace=True)

TableGeneral['%_quantity'] = (TableGeneral['quantity'])/(TableGeneral['total_quantity'])*100

TableGeneral['cumsum_quantity'] =  TableGeneral['%_quantity'].cumsum()
print(TableGeneral)

     stock_code                          description  price  quantity  \
4032      84077    WORLD WAR 2 GLIDERS ASSTD DESIGNS      0    105185   
4751     85123A   WHITE HANGING HEART T-LIGHT HOLDER  10068     91757   
3284      23843          PAPER CRAFT , LITTLE BIRDIE      2     80995   
4456      84879        ASSORTED COLOUR BIRD ORNAMENT   2692     78234   
2782      23166       MEDIUM CERAMIC TOP STORAGE JAR    198     77916   
...         ...                                  ...    ...       ...   
3796     72752A  F.FAIRY,CANDLE IN GLASS,LILY/VALLEY      4         1   
3804     72754D    F FAIRY POTPOURRI CUSHIONS SUMMER      1         1   
4672      85043           RED HEART CANDY POP LIGHTS      4         1   
1266      21852                         EMPIRE TEDDY      6         1   
3414      35950  FOLKART HEART CHRISTMAS DECORATIONS      1         1   

      total_price   %_price  cumsum_price revenue  total_quantity  %_quantity  \
4032      2044788  0.000000    100.000000 

In [86]:
def quantity(x):
    # допиши функцию
    if x <= 80:
        return 'A'
    elif x in range(80,95):
        return 'B'
    else:
        return 'C' 

# создай новый столбец popularity с помощью фу
TableGeneral['popularity'] = TableGeneral['cumsum_quantity'].apply(quantity) 
TableGeneral

,stock_code,description,price,quantity,total_price,%_price,cumsum_price,revenue,total_quantity,%_quantity,cumsum_quantity,popularity
4032,84077,WORLD WAR 2 GLIDERS ASSTD DESIGNS,0,105185,2044788,0.000000,100.000000,C,10528705,0.999031,0.999031,A
4751,85123A,WHITE HANGING HEART T-LIGHT HOLDER,10068,91757,2044788,0.492374,13.369112,A,10528705,0.871494,1.870524,A
3284,23843,"PAPER CRAFT , LITTLE BIRDIE",2,80995,2044788,0.000098,99.988116,C,10528705,0.769278,2.639802,A
4456,84879,ASSORTED COLOUR BIRD ORNAMENT,2692,78234,2044788,0.131652,38.919585,A,10528705,0.743054,3.382857,A
2782,23166,MEDIUM CERAMIC TOP STORAGE JAR,198,77916,2044788,0.009683,92.339401,C,10528705,0.740034,4.122891,A
...,...,...,...,...,...,...,...,...,...,...,...,...
3796,72752A,"F.FAIRY,CANDLE IN GLASS,LILY/VALLEY",4,1,2044788,0.000196,99.971440,C,10528705,0.000009,100.487657,C
3804,72754D,F FAIRY POTPOURRI CUSHIONS SUMMER,1,1,2044788,0.000049,99.998191,C,10528705,0.000009,100.487667,C
4672,85043,RED HEART CANDY POP LIGHTS,4,1,2044788,0.000196,99.971244,C,10528705,0.000009,100.487676,C
1266,21852,EMPIRE TEDDY,6,1,2044788,0.000293,99.924540,C,10528705,0.000009,100.487686,C


## Задание 6

__Объедини группы и проанализируй__

1. Создай серию `group` с объединенным столбцом отметок популярности и доходности товаров
2. Посчитай количество вхождений каждой сгруппированной пары
3. Какие группы непопулярны и приносят мало дохода?
4. На каких группах товаров нужно сфокусироваться при составлении плана закупок?

In [87]:
# напиши здесь код и комментарии
TableGeneral['group'] = TableGeneral['revenue'] + TableGeneral['popularity']
TableGeneral.head()

,stock_code,description,price,quantity,total_price,%_price,cumsum_price,revenue,total_quantity,%_quantity,cumsum_quantity,popularity,group
4032,84077,WORLD WAR 2 GLIDERS ASSTD DESIGNS,0,105185,2044788,0.000000,100.000000,C,10528705,0.999031,0.999031,A,CA
4751,85123A,WHITE HANGING HEART T-LIGHT HOLDER,10068,91757,2044788,0.492374,13.369112,A,10528705,0.871494,1.870524,A,AA
3284,23843,"PAPER CRAFT , LITTLE BIRDIE",2,80995,2044788,0.000098,99.988116,C,10528705,0.769278,2.639802,A,CA
4456,84879,ASSORTED COLOUR BIRD ORNAMENT,2692,78234,2044788,0.131652,38.919585,A,10528705,0.743054,3.382857,A,AA
2782,23166,MEDIUM CERAMIC TOP STORAGE JAR,198,77916,2044788,0.009683,92.339401,C,10528705,0.740034,4.122891,A,CA


In [88]:
popularity_dict = {"A": "наиболее ценные", 'B': "промежуточные", 'C': "наименее ценные",}
revenue_dict = {"A": "80% — продаж", 'B': "15% — продаж", 'C': "5% — продаж",}

In [89]:
TableGeneral['revenue'] = TableGeneral['revenue'].map(revenue_dict)
TableGeneral['popularity'] = TableGeneral['popularity'].map(popularity_dict)

TableGeneral['group_description'] =  TableGeneral['revenue'] + ' ' + TableGeneral['popularity']
TableGeneral

,stock_code,description,price,quantity,total_price,%_price,cumsum_price,revenue,total_quantity,%_quantity,cumsum_quantity,popularity,group,group_description
4032,84077,WORLD WAR 2 GLIDERS ASSTD DESIGNS,0,105185,2044788,0.000000,100.000000,5% — продаж,10528705,0.999031,0.999031,наиболее ценные,CA,5% — продаж наиболее ценные
4751,85123A,WHITE HANGING HEART T-LIGHT HOLDER,10068,91757,2044788,0.492374,13.369112,80% — продаж,10528705,0.871494,1.870524,наиболее ценные,AA,80% — продаж наиболее ценные
3284,23843,"PAPER CRAFT , LITTLE BIRDIE",2,80995,2044788,0.000098,99.988116,5% — продаж,10528705,0.769278,2.639802,наиболее ценные,CA,5% — продаж наиболее ценные
4456,84879,ASSORTED COLOUR BIRD ORNAMENT,2692,78234,2044788,0.131652,38.919585,80% — продаж,10528705,0.743054,3.382857,наиболее ценные,AA,80% — продаж наиболее ценные
2782,23166,MEDIUM CERAMIC TOP STORAGE JAR,198,77916,2044788,0.009683,92.339401,5% — продаж,10528705,0.740034,4.122891,наиболее ценные,CA,5% — продаж наиболее ценные
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3796,72752A,"F.FAIRY,CANDLE IN GLASS,LILY/VALLEY",4,1,2044788,0.000196,99.971440,5% — продаж,10528705,0.000009,100.487657,наименее ценные,CC,5% — продаж наименее ценные
3804,72754D,F FAIRY POTPOURRI CUSHIONS SUMMER,1,1,2044788,0.000049,99.998191,5% — продаж,10528705,0.000009,100.487667,наименее ценные,CC,5% — продаж наименее ценные
4672,85043,RED HEART CANDY POP LIGHTS,4,1,2044788,0.000196,99.971244,5% — продаж,10528705,0.000009,100.487676,наименее ценные,CC,5% — продаж наименее ценные
1266,21852,EMPIRE TEDDY,6,1,2044788,0.000293,99.924540,5% — продаж,10528705,0.000009,100.487686,наименее ценные,CC,5% — продаж наименее ценные


In [90]:
group_counts = TableGeneral.groupby('group').sum().reset_index()
group_counts.head()

,group,price,quantity,total_price,%_price,cumsum_price,total_quantity,%_quantity,cumsum_quantity
0,AA,970648,3433555,826094352,47.469371,22957.939552,4253596820,32.611371,21652.309159
1,AC,665124,385025,916065024,32.527773,27646.796343,4716859840,3.656907,41763.209777
2,CA,84259,4988048,1521322272,4.120672,71217.593609,7833356520,47.375703,44933.876008
3,CC,324757,1773425,7604566572,15.882184,360204.758733,39156253895,16.843714,359302.142134


In [91]:
group_counts = TableGeneral.groupby('group_description').sum().reset_index()
group_counts.head()

,group_description,price,quantity,total_price,%_price,cumsum_price,total_quantity,%_quantity,cumsum_quantity
0,5% — продаж наиболее ценные,84259,4988048,1521322272,4.120672,71217.593609,7833356520,47.375703,44933.876008
1,5% — продаж наименее ценные,324757,1773425,7604566572,15.882184,360204.758733,39156253895,16.843714,359302.142134
2,80% — продаж наиболее ценные,970648,3433555,826094352,47.469371,22957.939552,4253596820,32.611371,21652.309159
3,80% — продаж наименее ценные,665124,385025,916065024,32.527773,27646.796343,4716859840,3.656907,41763.209777


In [92]:
#Какие группы непопулярны и приносят мало дохода?

#Наименее популярной является группа АА.Самый низкий доход по группам АА и АС

In [93]:
#На каких группах товаров нужно сфокусироваться при составлении плана закупок?

#

## Задание 7

__Выгрузи данные по товарам с метокой по ABC-анализу__

1. Объедини по `stock_code` в одной таблице `ABC-код`, описание товара и `ID `

2. Выгрузи таблицу в `csv` для отправки в отдел закупок

In [107]:
TableGeneral_copy = TableGeneral.copy() # создадим новый датафрейм с копией исходного

In [108]:
TableGeneral_copy = pd.merge(TableGeneral_copy, 
                     products, 
                     left_on='stock_code', 
                     right_on='stock_code', 
                     how='inner')

In [109]:
TableGeneral_copy

,stock_code,description_x,price_x,quantity_x,total_price,%_price,cumsum_price,revenue,total_quantity,%_quantity,cumsum_quantity,popularity,group,group_description,invoice_id,description_y,quantity_y,price_y,total
0,84077,WORLD WAR 2 GLIDERS ASSTD DESIGNS,0,105185,2044788,0.000000,100.000000,5% — продаж,10528705,0.999031,0.999031,наиболее ценные,CA,5% — продаж наиболее ценные,489816,WORLD WAR 2 GLIDERS ASSTD DESIGNS,48,0,2044788
1,84077,WORLD WAR 2 GLIDERS ASSTD DESIGNS,0,105185,2044788,0.000000,100.000000,5% — продаж,10528705,0.999031,0.999031,наиболее ценные,CA,5% — продаж наиболее ценные,489819,WORLD WAR 2 GLIDERS ASSTD DESIGNS,96,0,2044788
2,84077,WORLD WAR 2 GLIDERS ASSTD DESIGNS,0,105185,2044788,0.000000,100.000000,5% — продаж,10528705,0.999031,0.999031,наиболее ценные,CA,5% — продаж наиболее ценные,490099,WORLD WAR 2 GLIDERS ASSTD DESIGNS,48,0,2044788
3,84077,WORLD WAR 2 GLIDERS ASSTD DESIGNS,0,105185,2044788,0.000000,100.000000,5% — продаж,10528705,0.999031,0.999031,наиболее ценные,CA,5% — продаж наиболее ценные,490100,WORLD WAR 2 GLIDERS ASSTD DESIGNS,2,0,2044788
4,84077,WORLD WAR 2 GLIDERS ASSTD DESIGNS,0,105185,2044788,0.000000,100.000000,5% — продаж,10528705,0.999031,0.999031,наиболее ценные,CA,5% — продаж наиболее ценные,490140,WORLD WAR 2 GLIDERS ASSTD DESIGNS,6,0,2044788
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1009940,72752A,"F.FAIRY,CANDLE IN GLASS,LILY/VALLEY",4,1,2044788,0.000196,99.971440,5% — продаж,10528705,0.000009,100.487657,наименее ценные,CC,5% — продаж наименее ценные,490360,"F.FAIRY,CANDLE IN GLASS,LILY/VALLEY",1,4,2044788
1009941,72754D,F FAIRY POTPOURRI CUSHIONS SUMMER,1,1,2044788,0.000049,99.998191,5% — продаж,10528705,0.000009,100.487667,наименее ценные,CC,5% — продаж наименее ценные,489822,F FAIRY POTPOURRI CUSHIONS SUMMER,1,1,2044788
1009942,85043,RED HEART CANDY POP LIGHTS,4,1,2044788,0.000196,99.971244,5% — продаж,10528705,0.000009,100.487676,наименее ценные,CC,5% — продаж наименее ценные,493536,RED HEART CANDY POP LIGHTS,1,4,2044788
1009943,21852,EMPIRE TEDDY,6,1,2044788,0.000293,99.924540,5% — продаж,10528705,0.000009,100.487686,наименее ценные,CC,5% — продаж наименее ценные,492750,EMPIRE TEDDY,1,6,2044788


In [113]:
TableGeneral_finish =TableGeneral_copy[['stock_code','group','description_x']]
TableGeneral_finish

,stock_code,group,description_x
0,84077,CA,WORLD WAR 2 GLIDERS ASSTD DESIGNS
1,84077,CA,WORLD WAR 2 GLIDERS ASSTD DESIGNS
2,84077,CA,WORLD WAR 2 GLIDERS ASSTD DESIGNS
3,84077,CA,WORLD WAR 2 GLIDERS ASSTD DESIGNS
4,84077,CA,WORLD WAR 2 GLIDERS ASSTD DESIGNS
...,...,...,...
1009940,72752A,CC,"F.FAIRY,CANDLE IN GLASS,LILY/VALLEY"
1009941,72754D,CC,F FAIRY POTPOURRI CUSHIONS SUMMER
1009942,85043,CC,RED HEART CANDY POP LIGHTS
1009943,21852,CC,EMPIRE TEDDY


In [115]:
TableGeneral_finish.to_csv(r"C:/Users/Xenia/Desktop/Учёба/Python/10 Удаленные БД/таблица_на_выгрузку.csv", index=False, sep=";")

# 🎉🎉🎉 Поздравляем, ты великолепен! 